# Módulo 1 - Cuál es tu mascota

En este módulo construiremos nuestro primer clasificador de imágenes desde cero y veremos si podemos lograr resultados de clase mundial.

Cada notebook comienza con las siguientes tres líneas; ellas aseguran que las ediciones en las bibliotecas que hagas se vuelvan a cargar aquí automáticamente, y también que se muestren en este notebook todos los cuadros o imágenes.

In [ ]:
import time
start_time = time.time()

In [ ]:
!pip install matplotlib

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Importamos todos los paquetes necesarios. Vamos a trabajar con la [biblioteca fastai V1] (http://www.fast.ai/2018/10/02/fastai-ai/) que se encuentra sobre [Pytorch 1.0] (https: // hackernoon. com / pytorch-1-0-468332ba5163). La biblioteca fastai proporciona muchas funciones útiles que nos permiten construir redes neuronales de manera rápida y fácil y entrenar nuestros modelos.

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

Si estás utilizando una computadora con una GPU inusualmente pequeña, puedes obtener un error de falta de memoria al ejecutar este notebook. Si esto sucede, haz clic en Kernel-> Reiniciar, elimina el comentario de la segunda línea a continuación para usar un *batch size* más pequeño (aprenderás todo sobre lo que esto significa durante el curso) e intenta nuevamente.

In [ ]:
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

## Mirando los datos

Vamos a utilizar el [Oxford-IIIT Pet Dataset] (http://www.robots.ox.ac.uk/~vgg/data/pets/) por [O. M. Parkhi et al., 2012] (http://www.robots.ox.ac.uk/~vgg/publications/2012/parkhi12a/parkhi12a.pdf) que presenta 12 razas de gatos y 25 razas de perros. Nuestro modelo necesitará aprender a diferenciar entre estas 37 categorías distintas. Según este artículo, la mejor precisión que pudieron obtener en 2012 fue del 59,21%, utilizando un modelo complejo que era específico para la detección de mascotas, con modelos separados de "Imagen", "Cabeza" y "Cuerpo" para las fotos de las mascotas. ¡Veamos qué tan precisos podemos ser usando el aprendizaje profundo (deep learning)!

Vamos a utilizar la función `untar_data` a la que debemos pasar una URL como argumento y que descargará y extraerá los datos.

In [ ]:
help(untar_data)

In [ ]:
path = untar_data(URLs.PETS); path

In [ ]:
path.ls()

In [ ]:
path_anno = path/'annotations'
path_img = path/'images'

Lo primero que hacemos cuando nos acercamos a un problema es echar un vistazo a los datos. _Siempre_ debemos entender muy bien cuál es el problema y cómo se ven los datos antes de que podamos descubrir cómo resolverlo. Echar un vistazo a los datos significa comprender cómo están estructurados los directorios de datos, cuáles son las etiquetas y cómo se ven algunas imágenes de muestra.

La principal diferencia entre el manejo de los conjuntos de datos de clasificación de imágenes es la forma en que se almacenan las etiquetas. En este conjunto de datos en particular, las etiquetas se almacenan en los propios nombres de archivo. Tendremos que extraerlos para poder clasificar las imágenes en las categorías correctas. Afortunadamente, la biblioteca fastai tiene una función práctica hecha exactamente para esto, `ImageDataBunch.from_name_re` obtiene las etiquetas de los nombres de archivo usando una [expresión regular] (https://docs.python.org/3.6/library/re.html).



In [ ]:
fnames = get_image_files(path_img)
fnames[:5]

In [ ]:
np.random.seed(2)
pat = r'/([^/]+)_\d+.jpg$'

In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs
                                  ).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

## Entrenamiento: resnet34

Ahora comenzaremos a entrenar nuestro modelo. Usaremos una [red neuronal convolucional] (http://cs231n.github.io/convolutional-networks/) troncal y un cabezal completamente conectado con una sola capa oculta como clasificador. ¿No sabes lo que significan estas cosas? No te preocupes, profundizaremos en los próximos módulos. Por el momento, necesitas saber que estamos construyendo un modelo que tomará imágenes como entrada y generará la probabilidad predicha para cada una de las categorías (en este caso, tendrá 37 salidas).

Entrenaremos durante 4 épocas (4 ciclos a través de todos nuestros datos).

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

## Resultados

Veamos qué resultados tenemos.

Primero veremos cuáles fueron las categorías que el modelo confundió más entre sí. Intentaremos ver si lo que predijo el modelo fue razonable o no. En este caso, los errores parecen razonables (ninguno de los errores parece obviamente ingenuo). Este es un indicador de que nuestro clasificador funciona correctamente.

Además, cuando graficamos la matriz de confusión, podemos ver que la distribución está muy sesgada: el modelo comete los mismos errores una y otra vez, pero rara vez confunde otras categorías. Esto sugiere que solo le resulta difícil distinguir algunas categorías específicas entre sí; Este es un comportamiento normal.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
doc(interp.plot_top_losses)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

## Descongelación, puesta a punto y tasas de aprendizaje

Dado que nuestro modelo funciona como esperamos, *descongelaremos* nuestro modelo y entrenaremos un poco más.

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.load('stage-1');

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

That's a pretty accurate model!

## Entrenamiento: resnet50

Ahora entrenaremos de la misma manera que antes pero con una advertencia: en lugar de usar resnet34 como nuestra red troncal, usaremos resnet50 (resnet34 es una red residual de 34 capas mientras que resnet50 tiene 50 capas. Se explicará más adelante en el curso y puedes conocer los detalles en el [documento de resnet] (https://arxiv.org/pdf/1512.03385.pdf)).

Básicamente, resnet50 generalmente funciona mejor porque es una red más profunda con más parámetros. Veamos si podemos lograr un mayor rendimiento aquí. Para ayudar a la red, también usaremos imágenes más grandes, ya que de esa manera la red puede ver más detalles. Reducimos un poco el tamaño del lote, ya que de lo contrario esta red más grande requerirá más memoria de GPU.

In [ ]:
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(),
                                   size=299, bs=bs//2).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(8)

In [ ]:
learn.save('stage-1-50')

¡Es sorprendente que sea posible reconocer las razas de mascotas con tanta precisión! Veamos si el ajuste completo ayuda:

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

Si no es así, siempre puedes volver a tu modelo anterior.

In [ ]:
learn.load('stage-1-50');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.most_confused(min_val=2)

## Otros formatos de datos

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE); path

In [ ]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_folder(path, ds_tfms=tfms, size=26)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy)
learn.fit(2)

In [ ]:
df = pd.read_csv(path/'labels.csv')
df.head()

In [ ]:
data = ImageDataBunch.from_csv(path, ds_tfms=tfms, size=28)

In [ ]:
data.show_batch(rows=3, figsize=(5,5))
data.classes

In [ ]:
data = ImageDataBunch.from_df(path, df, ds_tfms=tfms, size=24)
data.classes

In [ ]:
fn_paths = [path/name for name in df['name']]; fn_paths[:2]

In [ ]:
pat = r"/(\d)/\d+\.png$"
data = ImageDataBunch.from_name_re(path, fn_paths, pat=pat, ds_tfms=tfms, size=24)
data.classes

In [ ]:
data = ImageDataBunch.from_name_func(path, fn_paths, ds_tfms=tfms, size=24,
        label_func = lambda x: '3' if '/3/' in str(x) else '7')
data.classes

In [ ]:
labels = [('3' if '/3/' in str(x) else '7') for x in fn_paths]
labels[:5]

In [ ]:
data = ImageDataBunch.from_lists(path, fn_paths, labels=labels, ds_tfms=tfms, size=24)
data.classes

In [ ]:
elapsed_time = time.time() - start_time
print(elapsed_time)